# IRFs and sensitivity pyirf - Boostrap

**WARNING**

This is still a work-in-progress, it will evolve with the pipeline comparisons and converge with ctaplot+cta-benchmarks.

**IMPORTANT**

Soon this will be supersided by new results using [pyirf](https://github.com/cta-observatory/pyirf) and ctaplot metrics.

**Author(s):**
    
- Dr. Michele Peresano (CEA-Saclay/IRFU/DAp/LEPCHE), 2020
based on previous work by J. Lefacheur.
- Alice Donini (INFN Sezione di Trieste and Universita degli Studi di Udine), 2020
- Gaia Verna (Aix Marseille Univ, CNRS/IN2P3, CPPM, Marseille, France), 2020

based on [pyirf](https://github.com/cta-observatory/pyirf/blob/master/docs/notebooks/) 

**Description:**

This notebook contains DL3 and benchmarks for the _protopipe_ pipeline. 

Note that:
    - a more general set of benchmarks is being defined in cta-benchmarks/ctaplot,
    - follow [this](https://www.overleaf.com/16933164ghbhvjtchknf) document by adding new benchmarks or proposing new ones.

**Requirements:**

To run this notebook you will need a set of DL2 files produced on the grid with protopipe script _make_performance_pyirf_EventDisplay.py_

The MC production to be used and the appropriate set of files to use for this notebook can be found [here](https://forge.in2p3.fr/projects/step-by-step-reference-mars-analysis/wiki#The-MC-sample ).

The data format required to run the notebook is the current one used by _protopipe_ .

**Development and testing:**  

For the moment this notebook is optimized to work only on files produced from LSTCam + NectarCam telescope configurations.  
As with any other part of _protopipe_ and being part of the official repository, this notebook can be further developed by any interested contributor.  
The execution of this notebook is not currently automatic, it must be done locally by the user - preferably _before_ pushing a pull-request.  
**IMPORTANT:** Please, if you wish to contribute to this notebook, before pushing anything to your branch (better even before opening the PR) clear all the output and remove any local directory paths that you used for testing (leave empty strings).

**TODO:**  
* update everything...

## Table of contents

* [Optimized cuts](#Optimized-cuts)
    - [Direction cut](#Direction-cut)
* [Differential sensitivity from cuts optimization](#Differential-sensitivity-from-cuts-optimization)
* [IRFs](#IRFs)
    - [Effective area](#Effective-area)
    - [Point Spread Function](#Point-Spread-Function)
        + [Angular resolution](#Angular-resolution)
    - [Energy dispersion](#Energy-dispersion)
        + [Energy resolution](#Energy-resolution)
    - [Background rate](#Background-rate)

## Imports

In [ ]:
import os

import pyirf
import numpy as np
from astropy.io import fits
import uproot4 as uproot
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.table import QTable, Table, Column
from matplotlib.ticker import ScalarFormatter

from pyirf.binning import bin_center
from pyirf.utils import cone_solid_angle

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)

In [ ]:
Boostrap = np.arange(0,100)

## Input data

In [ ]:
protopipe_path = '/ctadata/gaia/PyIRF_data/IRFs_Sensitivity/Protopipe_Pyirf/STD_protopipe_NO/Baseline/New_RF/irf_tail_Time50.00h/'

# Performance calculated with pyirf from ED DL2 files
ED_pyirf_file = 'reference_data/aswg/point-like/LaPalma_20_South_EventDisplay/EventDisplay_pyirf_North_20deg_S_onaxis_50h.fits.gz'

### Load EventDisplay North perf

In [ ]:
# Path of EventDisplay IRF data in the user's local setup
# Please, empty the indir_EventDisplay variable before pushing to the repo
indir = "./reference_data/aswg/point-like/LaPalma_20_South_EventDisplay/"
irf_file_event_display = "DESY.d20191030.V2.ID0NIM2LST3MST3SST3SCMST3.prod3b-LaPalma-20degt05b-LL.Nb.3AL4-BN15.180000s.root"

irf_eventdisplay = uproot.open(os.path.join(indir, irf_file_event_display))

### Load Requirements

In [ ]:
indir = './reference_data/requirements/'

site = 'North'
obs_time = '50h'

# Full array
infiles = dict(sens=f'/{site}-{obs_time}.dat') # 30 min
requirements = dict()
for key in infiles.keys():
    requirements[key] = Table.read(indir + infiles[key], format='ascii')
requirements['sens'].add_column(Column(data=(10**requirements['sens']['col1']), name='ENERGY'))
requirements['sens'].add_column(Column(data=requirements['sens']['col2'], name='SENSITIVITY'))

## Optimized cuts

### Direction cut

In [ ]:
protopipe_file = protopipe_path + 'pyirf_protopipe.fits.gz'

rad_max = QTable.read(protopipe_file, hdu='RAD_MAX')[0]

energy_low = rad_max['ENERG_LO'].to_value(u.TeV)
energy_high = rad_max['ENERG_HI'].to_value(u.TeV)
energy_centers = 0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI']).to_value(u.TeV)
energy_width = 0.5 * (rad_max['ENERG_HI'] - rad_max['ENERG_LO']).to_value(u.TeV)

theta_cut_boostrap = {}
for e in  energy_centers:
    theta_cut_boostrap[e] = []   

In [ ]:
for i in Boostrap:
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    
    # [1:-1] removes under/overflow bins
    rad_max = QTable.read(protopipe_file, hdu='RAD_MAX')[0]

    for n, e in  enumerate(energy_centers):
        theta_cut_boostrap[e].append(rad_max['RAD_MAX'][0][n].value)

In [ ]:
for e in  energy_centers:
    
    print(f'Bin center {e} TeV')
    
    plt.hist(theta_cut_boostrap[e], bins=100, alpha=0.5, color='b')
    plt.title('theta_cut')
    plt.axvline(np.nanmean(theta_cut_boostrap[e]),color='r', linestyle='--',label= 'Mean = {:.3g}'.format(float(np.nanmean(theta_cut_boostrap[e]))))
    plt.legend()
    plt.grid()
    plt.show()    

In [ ]:
E = list(theta_cut_boostrap.keys())

theta_cut_boostrap_mean = [ np.nanmean(theta_cut_boostrap[e]) for e in E]
theta_cut_boostrap_std = [ np.nanstd(theta_cut_boostrap[e]) for e in E]

plt.errorbar(
    E,
    theta_cut_boostrap_mean,
    xerr = energy_width,
    yerr = theta_cut_boostrap_std,
)
plt.ylabel('θ-cut / deg')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.grid()

In [ ]:
protopipe_file = protopipe_path + 'pyirf_protopipe.fits.gz'

gh_cut = QTable.read(protopipe_file, hdu='GH_CUTS')[1:-1]

energy_low = gh_cut['low'].to_value(u.TeV)
energy_high = gh_cut['high'].to_value(u.TeV)
energy_centers = 0.5 * (gh_cut['low'] + gh_cut['high']).to_value(u.TeV)
energy_width = 0.5 * (gh_cut['high'] - gh_cut['low']).to_value(u.TeV)

gh_cut_boostrap = {}
for e in  energy_centers:
    gh_cut_boostrap[e] = [] 

In [ ]:
for i in Boostrap:
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    
    # [1:-1] removes under/overflow bins
    gh_cut = QTable.read(protopipe_file, hdu='GH_CUTS')[1:-1]

    for n, e in  enumerate(energy_centers):
        gh_cut_boostrap[e].append(gh_cut['cut'][n])

In [ ]:
for e in  energy_centers:
    
    print(f'Bin center {e} TeV')
    
    plt.hist(gh_cut_boostrap[e], bins=100, alpha=0.5, color='b')
    plt.title('gammaness_cut')
    plt.axvline(np.nanmean(gh_cut_boostrap[e]),color='r', linestyle='--',label= 'Mean = {:.3g}'.format(float(np.nanmean(gh_cut_boostrap[e]))))
    plt.legend()
    plt.grid()
    plt.show()    

In [ ]:
E = list(gh_cut_boostrap.keys())

gh_cut_boostrap_mean = [ np.nanmean(gh_cut_boostrap[e]) for e in E]
gh_cut_boostrap_std = [ np.nanstd(gh_cut_boostrap[e]) for e in E]

plt.errorbar(
    E,
    gh_cut_boostrap_mean,
    xerr = energy_width,
    yerr = gh_cut_boostrap_std,
)
plt.ylabel('gh-cut / deg')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.grid()

## Differential sensitivity from cuts optimization

In [ ]:
# [1:-1] removes under/overflow bins
sensitivity_protopipe = QTable.read(protopipe_file, hdu='SENSITIVITY')[1:-1]
energy_width = (sensitivity_protopipe['reco_energy_high'] - sensitivity_protopipe['reco_energy_low'])

#n_signal_boostrap = {}
#n_signal_w_boostrap = {}
#n_bkg_boostrap = {}
#n_bkg_w_boostrap = {}
flux_sensitivity_boostrap = {}

for e in energy_centers:
    flux_sensitivity_boostrap[e] = []

In [ ]:
for i in Boostrap:
    
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    
    # [1:-1] removes under/overflow bins
    sensitivity = QTable.read(protopipe_file, hdu='SENSITIVITY')[1:-1]

    for n, e in  enumerate(energy_centers):
        flux_sensitivity_boostrap[e].append(sensitivity['flux_sensitivity'][n].value)

In [ ]:
for e in  energy_centers[1:-2]:
    
    print(f'Bin center {e} TeV')
    
    plt.hist(flux_sensitivity_boostrap[e], bins=100, alpha=0.5, color='b')
    plt.title('theta_cut')
    plt.axvline(np.nanmean(flux_sensitivity_boostrap[e]),color='r', linestyle='--',label= 'Mean = {:.3g}'.format(float(np.nanmean(flux_sensitivity_boostrap[e]))))
    plt.legend()
    plt.grid()
    plt.show()    

In [ ]:
E = sensitivity_protopipe['reco_energy_center']

flux_sensitivity_mean = [ np.nanmean(flux_sensitivity_boostrap[e]) for e in E.value]
flux_sensitivity_std = [ np.nanstd(flux_sensitivity_boostrap[e]) for e in E.value]

unit = u.Unit('erg cm-2 s-1')

s = (E**2 * (flux_sensitivity_mean * u.Unit('cm-2 s-1 TeV-1')))
s_err = (E**2 * (flux_sensitivity_std * u.Unit('cm-2 s-1 TeV-1')))

# protopipe
plt.errorbar(E.value,
             s.to_value(unit),
             xerr = energy_width.value / 2,
             yerr = s_err.to_value(unit),
             label='protopipe'
            )
# Add requirements
plt.plot(requirements['sens']['ENERGY'], 
         requirements['sens']['SENSITIVITY'], 
         color='black', 
         ls='--', 
         lw=2, 
         label='Requirements'
)

plt.ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()

## Sensitivity comparison

In [ ]:
plt.figure(figsize=(12,8))
fig, (ax_sens, ax_ratio) = plt.subplots(
    2, 1,
    gridspec_kw={'height_ratios': [4, 1]},
    sharex=True,
)

E = sensitivity_protopipe['reco_energy_center']

flux_sensitivity_mean = [ np.nanmean(flux_sensitivity_boostrap[e]) for e in E.value]
flux_sensitivity_std = [ np.nanstd(flux_sensitivity_boostrap[e]) for e in E.value]

unit = u.Unit('erg cm-2 s-1')

s = (E**2 * (flux_sensitivity_mean * u.Unit('cm-2 s-1 TeV-1')))
s_err = (E**2 * (flux_sensitivity_std * u.Unit('cm-2 s-1 TeV-1')))

# protopipe
ax_sens.errorbar(E.value,
             s.to_value(unit),
             xerr = energy_width.value / 2,
             yerr = s_err.to_value(unit),
             label='protopipe',
             ls=''
            )
# ED
(y, yerr), edges = irf_eventdisplay["DiffSens"].to_numpy(errors=True)
bins = 10**edges
x = bin_center(bins)
width = np.diff(bins)
ax_sens.errorbar(
    x,
    y, 
    xerr=width/2,
    yerr=yerr,
    label="EventDisplay",
    ls=''
)

ax_ratio.errorbar(
    E.to_value(u.TeV), 
    s.to_value(unit) / y,
    xerr= energy_width.value /2,
    ls=''
)
ax_ratio.axhline(1, color = 'DarkOrange')
ax_ratio.set_yscale('log')
ax_ratio.set_xlabel("Reconstructed energy / TeV")
ax_ratio.set_ylabel('protopipe / eventdisplay')
ax_ratio.grid()
ax_ratio.yaxis.set_major_formatter(ScalarFormatter())

ax_ratio.set_ylim(0.5, 2.0)
ax_ratio.set_yticks([0.5, 2/3, 1, 3/2, 2])
ax_ratio.set_yticks([], minor=True)

# Style settings
ax_sens.set_title('Minimal Flux Satisfying Requirements for 50 hours')
ax_sens.set_xscale("log")
ax_sens.set_yscale("log")
ax_sens.set_ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")

ax_sens.grid(which="both")
ax_sens.legend()
fig.tight_layout(h_pad=0)

# IRFs

## Effective area

In [ ]:
area = QTable.read(protopipe_file, hdu='EFFECTIVE_AREA')[0]

energy_low = area['ENERG_LO'].to_value(u.TeV)
energy_high = area['ENERG_HI'].to_value(u.TeV)
energy_centers = 0.5 * (area['ENERG_LO'] + area['ENERG_HI']).to_value(u.TeV)[1:-1]
energy_width = 0.5 * (area['ENERG_LO'] - area['ENERG_HI']).to_value(u.TeV)[1:-1]

In [ ]:
Aeff_allcuts_boostrap = {}
Aeff_nocuts_boostrap = {}
Aeff_onlygh_boostrap = {}
Aeff_onlytheta_boostrap = {}

for e in  energy_centers:
    Aeff_allcuts_boostrap[e] = []
    Aeff_nocuts_boostrap[e] = []
    Aeff_onlygh_boostrap[e] = []
    Aeff_onlytheta_boostrap[e] = []

In [ ]:
for name in ('', '_NO_CUTS', '_ONLY_GH', '_ONLY_THETA'):
            
    for i in Boostrap:
        protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
        
        area = QTable.read(protopipe_file, hdu='EFFECTIVE_AREA' + name)[0]

        for n, e in  enumerate(energy_centers):
            if name == '':
                Aeff_allcuts_boostrap[e].append(area['EFFAREA'].to_value(u.m**2).T[1:-1, 0][n])
            elif name == '_NO_CUTS':
                Aeff_nocuts_boostrap[e].append(area['EFFAREA'].to_value(u.m**2).T[1:-1, 0][n])
            elif name == '_ONLY_GH':
                Aeff_onlygh_boostrap[e].append(area['EFFAREA'].to_value(u.m**2).T[1:-1, 0][n])
            elif name == '_ONLY_THETA':
                Aeff_onlytheta_boostrap[e].append(area['EFFAREA'].to_value(u.m**2).T[1:-1, 0][n])

In [ ]:
Aeff_mean=[]
Aeff_std=[]

name=''
Area = Aeff_allcuts_boostrap

for e in  energy_centers:
    print(f'Bin center {e} TeV')
    
    mean = np.nanmean(Area[e])
    std = np.nanstd(Area[e])
    
    Aeff_mean.append(mean)
    Aeff_std.append(std)
    
    plt.hist(Area[e],bins=100)
    plt.axvline(np.nanmean(Area[e]),
                color='r',linestyle='--',
                label= '{:.3g} +- {:.3g}'.format(mean,std))
    plt.axvline(mean+std,color='r',linestyle='-')
    plt.axvline(mean-std,color='r',linestyle='-')
    
    plt.title('Effective Area')
    print(f'NaN values encontered {sum(np.isnan(Area[e]))}')
    
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# protopipe
plt.errorbar(
    energy_centers,
    Aeff_mean,
    xerr=energy_width,
    yerr=Aeff_std,
    ls='',
    label='protopipe ' + name,
)

# ED
(y, yerr), edges = irf_eventdisplay["EffectiveAreaEtrue"].to_numpy(errors=True)
x = bin_center(10**edges)
xerr = 0.5 * np.diff(10**edges)
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")

# Style settings
plt.xscale("log")
plt.yscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Effective collection area / m²")
plt.grid(which="both")
plt.legend()

## Point Spread Function

In [ ]:
ang_res = QTable.read(protopipe_file, hdu='ANGULAR_RESOLUTION')[1:-1]

energy_low = ang_res['true_energy_low'].to_value(u.TeV)
energy_high = ang_res['true_energy_high'].to_value(u.TeV)
energy_centers = 0.5 * (ang_res['true_energy_low'] + ang_res['true_energy_high']).to_value(u.TeV)
energy_width = 0.5 * (ang_res['true_energy_high'] - ang_res['true_energy_low']).to_value(u.TeV)

In [ ]:
ang_res_boostrap = {}

for e in  energy_centers:
    ang_res_boostrap[e] = []

In [ ]:
for i in Boostrap:
    
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    
    ang_res = QTable.read(protopipe_file, hdu='ANGULAR_RESOLUTION')[1:-1]

    for n, e in  enumerate(energy_centers):
        ang_res_boostrap[e].append(ang_res['angular_resolution'][n].to_value())

In [ ]:
Angres_mean=[]
Angres_std=[]

for e in  energy_centers:
    print(f'Bin center {e} TeV')
    
    mean = np.nanmean(ang_res_boostrap[e])
    std = np.nanstd(ang_res_boostrap[e])
    
    Angres_mean.append(mean)
    Angres_std.append(std)
    
    plt.hist(ang_res_boostrap[e],bins=100)
    plt.axvline(np.nanmean(ang_res_boostrap[e]),
                color='r',linestyle='--',
                label= '{:.3g} +- {:.3g}'.format(mean,std))
    plt.axvline(mean+std,color='r',linestyle='-')
    plt.axvline(mean-std,color='r',linestyle='-')
    
    plt.title('Angular Resolution')
    print(f'NaN values encontered {sum(np.isnan(ang_res_boostrap[e]))}')
    
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# protopipe
plt.errorbar(
    energy_centers,
    Angres_mean,
    xerr=energy_width,
    yerr=Angres_std,
    ls='',
    label='pyirf'
)

# ED
# WARNING: ED's Angular resolution is plotted vs Ereco
(y, yerr), edges = irf_eventdisplay["AngRes"].to_numpy(errors=True)
x = bin_center(10**edges)
xerr = 0.5 * np.diff(10**edges)
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")


# Style settings
plt.xscale("log")
plt.yscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Angular Resolution / deg")
plt.grid(which="both")
plt.legend(loc="best")

None # to remove clutter by mpl objects

## Energy Resolution

In [ ]:
bias_resolution = QTable.read(protopipe_file, hdu='ENERGY_BIAS_RESOLUTION')[1:-1]

energy_low = bias_resolution['true_energy_low'].to_value(u.TeV)
energy_high = bias_resolution['true_energy_high'].to_value(u.TeV)
energy_centers = 0.5 * (bias_resolution['true_energy_low'] + bias_resolution['true_energy_high']).to_value(u.TeV)
energy_width = 0.5 * (bias_resolution['true_energy_high'] - bias_resolution['true_energy_low']).to_value(u.TeV)

In [ ]:
energy_res_boostrap = {}

for e in  energy_centers:
    energy_res_boostrap[e] = []

In [ ]:
for i in Boostrap:
    
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    energy_res = QTable.read(protopipe_file, hdu='ENERGY_BIAS_RESOLUTION')[1:-1]

    for n, e in  enumerate(energy_centers):
        energy_res_boostrap[e].append(energy_res['resolution'][n])

In [ ]:
Energyres_mean=[]
Energyres_std=[]

for e in  energy_centers:
    print(f'Bin center {e} TeV')
    
    mean = np.nanmean(energy_res_boostrap[e])
    std = np.nanstd(energy_res_boostrap[e])
    
    Energyres_mean.append(mean)
    Energyres_std.append(std)
    
    plt.hist(energy_res_boostrap[e],bins=100)
    plt.axvline(np.nanmean(energy_res_boostrap[e]),
                color='r',linestyle='--',
                label= '{:.3g} +- {:.3g}'.format(mean,std))
    plt.axvline(mean+std,color='r',linestyle='-')
    plt.axvline(mean-std,color='r',linestyle='-')
    
    plt.title('Angular Resolution')
    print(f'NaN values encontered {sum(np.isnan(ang_res_boostrap[e]))}')
    
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# protopipe
plt.errorbar(
    energy_centers,
    Energyres_mean,
    xerr=energy_width,
    yerr=Energyres_std,
    ls='',
    label='pyirf'
)

# ED
# WARNING: ED's Energy resolution is plotted vs Ereco
(y, yerr), edges = irf_eventdisplay["ERes"].to_numpy(errors=True)
x = bin_center(10**edges)
xerr = np.diff(10**edges) / 2
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")


# Style settings
plt.xlim(1.e-2, 2.e2)
plt.ylim(2.e-2, 1)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Energy Resolution / deg")
plt.grid(which="both")
plt.legend(loc="best")

None # to remove clutter by mpl objects

## Background rate

In [ ]:
from pyirf.utils import cone_solid_angle

# protopipe
bg_rate = QTable.read(protopipe_file, hdu='BACKGROUND')[0]

energy_low = bg_rate['ENERG_LO'].to_value(u.TeV)
energy_high = bg_rate['ENERG_HI'].to_value(u.TeV)
energy_centers = 0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI']).to_value(u.TeV)
energy_width = 0.5 * (bg_rate['ENERG_HI'] - bg_rate['ENERG_LO']).to_value(u.TeV)

In [ ]:
bg_rate_boostrap = {}

for e in  energy_centers:
    bg_rate_boostrap[e] = []

In [ ]:
for i in Boostrap:
    
    protopipe_file = protopipe_path + f'pyirf_protopipe_{i}.fits.gz'
    bg_rate = QTable.read(protopipe_file, hdu='BACKGROUND')[0]
    
    reco_bins = np.append(bg_rate['ENERG_LO'], bg_rate['ENERG_HI'][-1])

    # first fov bin, [0, 1] deg
    fov_bin = 0
    rate_bin = bg_rate['BKG'].T[:, fov_bin]

    # interpolate theta cut for given e reco bin
    e_center_bg = 0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI'])
    e_center_theta = 0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI'])
    theta_cut = np.interp(e_center_bg, e_center_theta, rad_max['RAD_MAX'].T[:, 0])

    # undo normalization
    rate_bin *= cone_solid_angle(theta_cut)
    rate_bin *= np.diff(reco_bins)

    for n, e in  enumerate(energy_centers):
        bg_rate_boostrap[e].append(rate_bin[n].to_value(1 / u.s))

In [ ]:
Bg_rate_mean=[]
Bg_rate_std=[]

for e in  energy_centers:
    print(f'Bin center {e} TeV')
    
    """if np.isnan(bg_rate_boostrap[e]).all():
        print('all NaN!')
    else:"""
    mean = np.nanmean(bg_rate_boostrap[e])
    std = np.nanstd(bg_rate_boostrap[e])


    Bg_rate_mean.append(mean)
    Bg_rate_std.append(std)

    plt.hist(bg_rate_boostrap[e],bins=100)
    plt.axvline(np.nanmean(bg_rate_boostrap[e]),
                color='r',linestyle='--',
                label= '{:.3g} +- {:.3g}'.format(mean,std))
    plt.axvline(mean+std,color='r',linestyle='-')
    plt.axvline(mean-std,color='r',linestyle='-')

    plt.title('Bkg Rate')
    print(f'NaN values encontered {sum(np.isnan(bg_rate_boostrap[e]))}')

    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# protopipe
plt.errorbar(
    energy_centers,
    Bg_rate_mean,
    xerr=energy_width,
    yerr=Bg_rate_std,
    ls='',
    label='pyirf'
)

# ED
(y, yerr), edges = irf_eventdisplay["BGRate"].to_numpy(errors=True)
x = bin_center(10**edges)
xerr = np.diff(10**edges) / 2
plt.errorbar(x, y, xerr=xerr, yerr=yerr, ls='', label="EventDisplay")



# Style settings
plt.xscale("log")
plt.xlabel(r"$E_\mathrm{Reco} / \mathrm{TeV}$")
plt.ylabel("Background rate / (s⁻¹ TeV⁻¹) ")
plt.grid(which="both")
plt.legend(loc="best")
plt.yscale('log')

None # to remove clutter by mpl objects